In [1]:
from models import Ensemble
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from dataset import LABELS

In [2]:
test_data = pd.read_pickle('extracted_features\\features_test.pkl')
test = tf.data.Dataset.from_tensor_slices((test_data, np.zeros(len(test_data))))

silence = pd.read_pickle('extracted_features\\silence_detected.pkl')

LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
LABELS_dict[10] = 'unknown'

In [3]:
model_paths = os.listdir('models\\ensemble\\')
model_paths = ['models\\ensemble\\' + path for path in model_paths]

In [4]:
ensemble = Ensemble(model_paths=model_paths)

In [5]:
pred = ensemble.predict_mean(test.batch(10))

15854/15854 [==============================] - 340s 21ms/step


In [6]:
pred

array([ 1, 10,  8, ...,  3,  3, 10], dtype=int64)

In [7]:
x = pd.concat([pd.DataFrame(pred), pd.DataFrame(silence)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [8]:
df = pd.read_csv('submissions\\sample_submission.csv', header = 0)
df['label'] = res
df.to_csv(f'submissions\\submission_ensemble_mean.csv', index=False, header=True)

In [ ]:
pred2 = ensemble.predict_max(test.batch(10))

In [ ]:
x = pd.concat([pd.DataFrame(pred2), pd.DataFrame(silence)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [ ]:
df = pd.read_csv('submissions\\sample_submission.csv', header = 0)
df['label'] = res
df.to_csv(f'submissions\\submission_ensemble_max.csv', index=False, header=True)